## Analysis - coming up with answers
We are now ready to come up with answers...

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

import pandas as pd
pd.options.plotting.backend = "plotly"

from tinydb import TinyDB, Query

import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import datetime as dt

from sklearn.preprocessing import minmax_scale



In [2]:
db = TinyDB('data\hogwarts.json')

cleaned_users = db.table('cleaned_users')
cleaned_actions = db.table('cleaned_actions') 

users_df = pd.DataFrame(cleaned_users.all())
actions_df = pd.DataFrame(cleaned_actions.all())

db.close()

In [3]:
combined_users_actions = pd.merge(users_df, actions_df, on="user_id", how="outer", validate= "1:m", indicator=True)
combined_users_actions.head()

,user_id,account_creation_timestamp,house,action_timestamp,action,_merge
0,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-10 03:02:59,preparing potion,both
1,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-05 17:21:30,preparing potion,both
2,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-05 16:55:38,show attendance to magic class,both
3,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-10 03:32:47,preparing potion,both
4,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-10 03:02:25,create new potion,both
...,...,...,...,...,...,...
168893,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:40,preparing potion,both
168894,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:40,preparing potion,both
168895,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:39,preparing potion,both
168896,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:39,preparing potion,both


In [4]:
## Due to a bug there is a user without any actions. We drop the user, and then the _merge column
combined_users_actions =  combined_users_actions[combined_users_actions['_merge'] == 'both' ]
combined_users_actions.drop(columns='_merge', inplace=True)


## We need to do a hack

When we prepared the data we observed that the action_timestamp had duplicate values. The choice to leave them as is, came back biting. The duplicate values for the same users, did not allow us to correctly count sessions for the 26 users that had two different actions performed at exactly the same time.

The motivations for keeping them is because our system keeps other records at the level of milliseconds. So it could be the case that these values are valid, and only seemingly duplicate. We should raise this issue with the data engineering team.

The solution is to add microseconds to the action_timestamp. We do that by utilising the dataframe index, that is guaranteed to be unique for each row. Incidentally, the index goes up to six digits, exactly what the datetime type can handle. 

In that way, all action timestamps are now unique, with no duplicates.

In [5]:
## We get the old action_timestamp and add unique integers - the index -> new action_timestamp with no duplicates

combined_users_actions['action_timestamp'] = \
    combined_users_actions.apply( lambda x : x['action_timestamp'] + '.' +str(x.name).zfill(6) , axis=1)

combined_users_actions.head()

In [7]:
## Like before we cast the string representation of date into datetime for the advanced capabilities of the type
combined_users_actions['account_ts_to_dt'] = combined_users_actions.account_creation_timestamp.apply(pd.to_datetime)

combined_users_actions['action_ts_to_dt'] = combined_users_actions.action_timestamp.apply(pd.to_datetime)

combined_users_actions.head()


,user_id,account_creation_timestamp,house,action_timestamp,action,account_ts_to_dt,action_ts_to_dt
0,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-10 03:02:59.000000,preparing potion,2021-05-05 11:26:07.505,2021-05-10 03:02:59.000000
1,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-05 17:21:30.000001,preparing potion,2021-05-05 11:26:07.505,2021-05-05 17:21:30.000001
2,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-05 16:55:38.000002,show attendance to magic class,2021-05-05 11:26:07.505,2021-05-05 16:55:38.000002
3,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-10 03:32:47.000003,preparing potion,2021-05-05 11:26:07.505,2021-05-10 03:32:47.000003
4,609280cfd65e4600a8be2f67,2021-05-05 11:26:07.505,Hufflepuff,2021-05-10 03:02:25.000004,create new potion,2021-05-05 11:26:07.505,2021-05-10 03:02:25.000004
...,...,...,...,...,...,...,...
168893,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:40.168893,preparing potion,2021-06-02 16:02:00.461,2021-06-02 18:08:40.168893
168894,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:40.168894,preparing potion,2021-06-02 16:02:00.461,2021-06-02 18:08:40.168894
168895,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:39.168895,preparing potion,2021-06-02 16:02:00.461,2021-06-02 18:08:39.168895
168896,60b7ab78dd93f900a8ce7e76,2021-06-02 16:02:00.461,Gryffindor,2021-06-02 18:08:39.168896,preparing potion,2021-06-02 16:02:00.461,2021-06-02 18:08:39.168896


In [8]:
## We'll start working with the time difference between registration and action - we start with each action separately.

combined_users_actions['time_diff_in_min'] = \
(   
    combined_users_actions.action_timestamp.apply(pd.to_datetime) - \
    combined_users_actions.account_creation_timestamp.apply(pd.to_datetime)

).apply( lambda x: round(x.total_seconds() / 60, 0) ) 

In [9]:
## We ask a question: how many users are active in the first two hours?
## A bit more than 1%

time_limit = 120

first_activity_data = combined_users_actions.query('time_diff_in_min < {} '.format(time_limit)).user_id.unique()
print("Number of active users in the first two hours after registration:",first_activity_data.size)

Number of active users in the first two hours after registration: 44


In [11]:
## We now turn to the activity per user. 
## We start with the maximum period of activity, defined as the time between registration and last action.

max_active_period_df = pd.DataFrame(
    combined_users_actions.groupby('user_id').apply(lambda x: round(
        (x.action_ts_to_dt.max() -  x.account_ts_to_dt.min() ).total_seconds() / 60, 0
        ) ) )

max_active_period_df = max_active_period_df.reset_index().rename(columns={0:'max_active_period_per_user'})

In [12]:
fig1_data =  max_active_period_df['max_active_period_per_user'].sort_values().reset_index().drop(columns='index')

fig1 = px.line( fig1_data,
    title='Time difference between a user\'s last action and registration, per user')

series_names = [ "Time difference in min" ]

for idx, name in enumerate(series_names):
    fig1.data[idx].name = name

fig1.show()

fig1.write_image("../presentation/figures/fig1.png")

### ------------COMMENT------------ ###

We see that for many users, the total period of activity, from registration to last action, lasts for approximately two hours. Recall though, that less than 2% have recorded an action within the first two hours

Let us take a closer look.

In [13]:
## We now present the histogram of the same data. We use two time limits to focus on the time range we want.

lower_limit = 0

upper_limit = 350


fig2_data = max_active_period_df['max_active_period_per_user']\
    .sort_values().reset_index().drop(columns='index')\
    .query('max_active_period_per_user > {} & max_active_period_per_user < {}'.format(lower_limit, upper_limit))

fig2 = px.histogram(fig2_data
        ,title='Time difference between a user\'s last action and registration, per user'
        ,nbins= int((upper_limit - lower_limit)/10)  )  ## bins are set to 10 minute intervals

series_names = [ "Time difference in min" ]

for idx, name in enumerate(series_names):
    fig2.data[idx].name = name

fig2.show()

fig2.write_image("../presentation/figures/fig2.png")


### ------------COMMENT------------ ###

We observe the following pattern: 
- More than half of the registered users (1821 out of 3346), stop using the application within 20 minutes after this two hour limit.
- We know that only less than 2% has registered an action in the first two hours.
- We conclude that for more than half of our users their active engagement with the application lasts at most 20 minutes

Let us explore the difference in the types of actions performed within and after the first two hours following registration.



In [14]:
## Same data as before, now split along the dimension of time

left_limit = 140

right_limit = 141

less_than = combined_users_actions.query(
    'time_diff_in_min < {} '.format(left_limit)
    ).groupby('action')[['user_id','action_timestamp']].count()

more_than = combined_users_actions.query(
    'time_diff_in_min > {} '.format(right_limit)
    ).groupby('action')[['user_id','action_timestamp']].count()

less_than['percent'] = (less_than['user_id'] / less_than['user_id'].sum()) * 100

more_than['percent'] = (more_than['user_id'] / more_than['user_id'].sum()) * 100

fig3 = go.Figure(
    data=[
        go.Bar(
            name="< {} min".format(left_limit),
            x=less_than.index,
            y=less_than["percent"],
            offsetgroup=0,
        ),
        go.Bar(
            name="> {} min".format(right_limit),
            x=more_than.index,
            y=more_than['percent'],
            offsetgroup=1,
        ),
    ],
    layout=go.Layout(
        title="Actions as percentage of total actions, for two groups. Num of actions A:{}, B:{} ".format(
            less_than.user_id.sum(),
            more_than.user_id.sum() 
            ),
        yaxis_title="Actions as percentage of total"
    )
)
fig3.show()

fig3.write_image("../presentation/figures/fig3.png")

### ------------COMMENT------------ ###

It seems that our users interact differently with the app, in the first 140 minutes after registration, compared to beyond 140 minutes. 
- The evidence suggest general interest during their first contact, with most actions relating to class activities - "ask for teaching assistant", "create potion", "review curriculum", etc.
- After that time limit, the major action that covers two thirds of all actions is "preparing potion". 
- One third of the total actions (51979 / 116245) performed by users, takes place in this "exploratory" phase.


In [15]:
## Let us see if there is any variation across the four houses

left_limit = 140

right_limit = 141

houses = ['Hufflepuff', 'Gryffindor', 'Slytherin', 'Ravenclaw']

for house in houses:

    less_than = combined_users_actions[combined_users_actions['house'] == \
         house].query('time_diff_in_min < {} '.format(left_limit)).groupby('action')[['user_id','action_timestamp']].count()

    more_than = combined_users_actions[combined_users_actions['house'] == \
        house].query('time_diff_in_min > {} '.format(right_limit)).groupby('action')[['user_id','action_timestamp']].count()

    less_than['percent'] = (less_than['user_id'] / less_than['user_id'].sum()) * 100

    more_than['percent'] = (more_than['user_id'] / more_than['user_id'].sum()) * 100

    fig4 = go.Figure(
        data=[
            go.Bar(
                name="< {} min".format(left_limit),
                x=less_than.index,
                y=less_than["percent"],
                offsetgroup=0,
            ),
            go.Bar(
                name="> {} min".format(right_limit),
                x=more_than.index,
                y=more_than['percent'],
                offsetgroup=1,
            ),
        ],
        layout=go.Layout(
            title="Actions as percentage of total actions, for two groups. House is {}. Num of actions A:{}, B:{} ".format(
                house
                ,less_than.user_id.sum()
                ,more_than.user_id.sum() 
                ),
            yaxis_title="Actions in percentage"
        )
    )
    fig4.show()

    fig4.write_image("../presentation/figures/{}.png".format(house) )



### ------------COMMENT------------ ###

There doesn't seem to be a difference, before and after the 140 minute limit for the four schools. The pattern of usage remains the same.

What we should note though, are the differences in the proportions of actions before and after 140 minutes for the four schools. Overall, one third of actions are performed in the "exploratory" phase. Instead, for Slytherin almost half of the actions are within 140 minutes, while for Ravenclaw this is only one quarter.

## Users, actions and sessions

We now turn to the 3 core questions we are give. For doing so we'll start with the concept of a session, because it will give some insight to be used for the first two. Besides, it is the most direct question, in the sense that it provides a clear metric to work with.

We've prepared a separate module (with a single function) for computing the sessions. A “ 'session' is defined as a length of time where a student performs a series of actions with less than 10 minutes in between them."

In [16]:
import module_session_duration as sd

## The docstring for the function we'll use in pd.DataFrame.apply( sd.find_session_duration() )

help(sd.find_session_duration)

Help on function find_session_duration in module module_session_duration:

find_session_duration(dates_array, minimum_duration=600, ignore_isolated=False, isolated_duration=60)
    Reads an array of timestamps and determines groups of them, when each differs from previous one for less than a minimum duration
    
    Arguments:
        dates_array: An pandas.Series of datetime objects
        minimun_duration: An int or float, denoting a time interval in seconds
        ignore_isolated: A bool flag, when false each action not in a session gets a user defined duration.
        isolated_duration: the user defined duration for actions outside of sessions. Active when ignore_isolated=False
    
    Returns:
        A pandas.Series of floats. Each item in the list, is the time elapsed between two datetimes in minutes.
    
    Special cases:
        Returns empty Series when 1) List length less than two, 2) not found any matching intervals
    
    Examples:
       Inputs: [Timestamp('2021-

In [17]:
## We first compute the sessions for 10 minute intervals

user_sessions_df = combined_users_actions.groupby('user_id').apply(
        lambda x: sd.find_session_duration(x.action_ts_to_dt, 60*10, ignore_isolated=False )
        ).reset_index().rename(columns={0: 'session_duration', 'level_1': 'session_rank'})

In [18]:
print(
"There are {} users that have recorded at least one session".format( 
    user_sessions_df.groupby('user_id').count().reset_index().last_valid_index() + 1) 
)

There are 3222 users that have recorded at least one session


### ------------COMMENT------------ ###

As a limited sanity check for our calculations, recall that there are 3345 user that we analyse. 123 of these users have only one recorded action, and our function does not provide session duration for them.

With the ignore_isolated=True it only gives us results back for 3186 users. That leaves us with 35 users (35 = 3345 -123 -3186) that are unaccounted for. This is due to the way we perform the computations. 

In contrast, when we select ignore_isolated=False and/or we set the session limit to a really large time period that covers the whole time span of our data, the reported users with at least one session become 3222 (everyone has at least one start and one end date), and all users are accounted for.

In [19]:
## Let's see the user with the longest session. 
user_sessions_df.sort_values('session_duration', ascending=False).head(10) 

,user_id,session_rank,session_duration
1340,609712e1d65e4600a8d0c166,26,154.45
4625,60a4f6c3b4078800a8230d5f,7,124.18
9219,60b3fdee6a426400a8131f2b,0,118.58
8422,60b095cda8a7fb00a8e1524e,1,116.57
966,6095919fbea2a400a89e84d4,5,110.05
6024,60a91fe31ff78a00a9f0376f,27,97.80
3202,60a0b4036500f400a914e22f,0,96.98
10729,60b8f11578b0e700a8ec818b,4,94.75
7151,60ad23ce0ddefb00a870a9c7,87,93.47
7662,60ae66c311f13500a8b79e98,21,93.10


## On question 3

The student with the longest uninterrupted session is 609712e1d65e4600a8d0c166 with a session that lasted for 154 minutes and 27 seconds.

A user's sessions are first, second, etc. This is session_rank. For our user, the longest session was the 26th in chronological order, out of around 40 (that do not appear here). 

## Let us now take a look at question 2. 

The task is to identify "the most active student of all Hogwarts in terms of total time spent performing
actions and total number of actions performed".

For question 3 we have already created a metric for the time spent performing actions - we created the sessions and we have chosen to add, a somewhat arbitrary, 1 minute for any isolated action outside a session. We just need to add the whole sessions' duration for a user and we will have the total time spent. We don't consider the option of simply time differencing from first to last action because that can be grossly misleading - imagine a single user with two actions months apart.

For counting the actions, we have a couple of options open to us.
- For each user we have counted the duration of a single session, but we also counted the number of sessions, including those single actions here and there more than 10 minutes apart from other actions. Some actions are lost as simple counts, because are grouped together, but their presence is reflected in the session duration.
- The alternative would be to use the straightforward count of actions per user. That might be preferable to the previous alternative, if for example revenue depended on individual user actions.

We'll a give a try to both alternatives, since we don't have strong reasons neither to accept nor to reject any of the two.

In [20]:
## We start by computing the number of total actions per user

actions_per_user_df =\
    combined_users_actions[['user_id', 'action']].\
    groupby('user_id').\
    count().\
    reset_index().\
    rename(columns={'action': 'total_actions_per_user'})

actions_per_user_df.head()

,user_id,total_actions_per_user
0,601c2dd884c5dec8773309ce,4
1,601c30a284c5dec8773315fd,51
2,601c69ea84c5dec8773401f1,1
3,6091f1e5bea2a400a88e93fc,20
4,6091f23cbea2a400a88e95c3,34


In [21]:
## We then compute the number of total sessions per user
## Watch out: session rank is zero based - rank 0 = 1 action

sessions_per_user_df = \
    user_sessions_df.\
    groupby('user_id')[['session_rank']].\
    max().\
    rename(columns={'session_rank':'total_sessions_per_user'}).\
    apply(lambda x: x+1).\
    reset_index()

sessions_per_user_df.head()

,user_id,total_sessions_per_user
0,601c2dd884c5dec8773309ce,1
1,601c30a284c5dec8773315fd,2
2,6091f1e5bea2a400a88e93fc,1
3,6091f23cbea2a400a88e95c3,2
4,6091f4d4bea2a400a88ea3cd,13


In [22]:
## Finally we want the total sessions' duration per user
duration_per_user_df = \
    user_sessions_df.\
    groupby('user_id')[['session_duration']].\
    sum().\
    rename(columns={'session_duration':'total_duration_per_user'}).\
    reset_index()

duration_per_user_df.head()

,user_id,total_duration_per_user
0,601c2dd884c5dec8773309ce,1.28
1,601c30a284c5dec8773315fd,12.05
2,6091f1e5bea2a400a88e93fc,3.08
3,6091f23cbea2a400a88e95c3,14.08
4,6091f4d4bea2a400a88ea3cd,104.58


In [23]:
total_activity = pd.merge( users_df[['user_id', 'house']]
                    , actions_per_user_df
                    , on="user_id", how="outer", validate= "1:1", indicator=True)

total_activity.drop('_merge', axis=1, inplace=True)

In [24]:
## We now collect all the pieces together, using the user_id as the key
## Since we will also need the School for question 1, we'll start with the users_df and bring School too.


total_activity = pd.merge( users_df[['user_id', 'house']]
                    , actions_per_user_df
                    , on="user_id", how="outer", validate= "1:1", indicator=True)          
total_activity.drop('_merge', axis=1, inplace=True)


total_activity = pd.merge( total_activity
                    , sessions_per_user_df
                    , on="user_id", how="outer", validate= "1:1", indicator=True)
total_activity.drop('_merge', axis=1, inplace=True)


total_activity = pd.merge( total_activity
                    , duration_per_user_df
                    , on="user_id", how="outer", validate= "1:1", indicator=True)


total_activity

,user_id,house,total_actions_per_user,total_sessions_per_user,total_duration_per_user,_merge
0,609280cfd65e4600a8be2f67,Hufflepuff,67.0,5.0,26.55,both
1,609bdfad6500f400a9ffa8ba,Gryffindor,48.0,4.0,11.45,both
2,60a4e57bb4078800a8226b22,Gryffindor,20.0,1.0,4.45,both
3,60a3269d1ff78a00a9ceef23,Gryffindor,1.0,NaN,NaN,left_only
4,60ad06b911f13500a8aeb5c9,Gryffindor,11.0,1.0,9.77,both
...,...,...,...,...,...,...
3341,60b3fdee6a426400a8131f2b,Hufflepuff,227.0,7.0,157.96,both
3342,60982190bea2a400a8a72657,Ravenclaw,13.0,1.0,2.73,both
3343,609c191bc6214800a8aa0100,Slytherin,55.0,1.0,28.63,both
3344,60b3bd3bc7002f00a87aa2f4,Ravenclaw,7.0,1.0,2.03,both


In [25]:
## It seems we brought everything together, without major mistakes.
print(" We have a total of 3346 rows, where {} are those with a single action, exactly as before".\
    format(total_activity[total_activity['_merge'] == 'left_only'].reset_index().last_valid_index() +1 )
)

total_activity = total_activity[total_activity['_merge'] == 'both']
total_activity.drop('_merge', axis=1, inplace=True)


 We have a total of 3346 rows, where 124 are those with a single action, exactly as before


In [26]:
fig5 = px.scatter_matrix(total_activity
    ,dimensions=['total_actions_per_user', 'total_sessions_per_user', 'total_duration_per_user']
    ,color="house"
    ,title="Scatter matrix for our activity metrics"
    ,labels={col:col.replace('_', ' ').replace('total ', '').replace(' per user', '') for col in total_activity.columns}
    )
fig5.show()

### ------------COMMENT------------ ###

- The tail to the right, for all figures, are dominated by Ravenclaw and Gryffindor.
- As we have seen before, the majority of data points are clustered to the far left, with extreme at the right. These are not necessarily outliers; they might as well be power users, while the whole distribution of each random variable could belong to the exponential distribution.
- The correlation between our metrics, are positively linear; slightly less so for sessions.

## How about our activity metric then?

We have one metric for total time spent, and two more for total actions performed. We now have have to combine them into one metric for total activity. A version of mean should suffice for our task

In the presence of a few large values, the arithmetic mean can be skewed towards these large values. For that reason we choose the harmonic mean, as way to produce a metric more resistant to extremely large values. In addition, we will compute weighted harmonic mean, in order to give our stakeholders the chance to manipulate this metric according their assigned relative importance for time and actions. You can think of it as a weighted F1 score of precision and recall.

One last issue is scaling. We will combine two different metrics that have different scales. That means that the one with the largest scale will dominate the results. In order to avoid that we will first scale the time and actions metrics. We chose a min-max scaling in order to avoid the mean and standard deviation, in the presence of large values. Still, small values might be compressed close to zero, but at least for questions about the "most" active, that should not be a problem.

In [27]:
time_weight = 0.5
actions_weight = 1 - time_weight

try:
    assert( (time_weight>=0) and (time_weight<=1) )
except AssertionError:
    time_weight = 0.5
    actions_weight = 1 - time_weight
    print("Please choose valid weights, between 0 and 1. \nFor now the weights have been back to default, 0.5 for each")

In [28]:
## We'll use that for the df.apply()
## We'll need a small hack - in some cases the duration is less than a second, and in the rounding it is returned as zero.
## We add a small constant to the denominators
## This is due to the duplicate actions that we chose to NOT to drop as duplicates.


weighted_harmonic = lambda duration,action : (time_weight + actions_weight) / ( (time_weight/(duration+0.01)) + (actions_weight/(action+0.01)) )

In [29]:
## 

total_activity['scaled_actions'] = minmax_scale(total_activity.total_actions_per_user)
total_activity['scaled_sessions'] = minmax_scale(total_activity.total_sessions_per_user)
total_activity['scaled_duration'] = minmax_scale(total_activity.total_duration_per_user)


In [30]:
## The two metrics

total_activity['duration_sessions_metric'] = \
    total_activity.\
    apply(
        lambda x: weighted_harmonic( x.scaled_duration, x.scaled_sessions ), axis=1
    )



total_activity['duration_actions_metric'] = \
    total_activity.\
    apply(
        lambda x: weighted_harmonic( x.scaled_duration, x.scaled_actions ), axis=1
    )
    

In [31]:
total_activity

,user_id,house,total_actions_per_user,total_sessions_per_user,total_duration_per_user,scaled_actions,scaled_sessions,scaled_duration,duration_sessions_metric,duration_actions_metric
0,609280cfd65e4600a8be2f67,Hufflepuff,67.0,5.0,26.55,0.021438,0.020202,0.020560,0.030380,0.030993
1,609bdfad6500f400a9ffa8ba,Gryffindor,48.0,4.0,11.45,0.015172,0.015152,0.008867,0.021560,0.021568
2,60a4e57bb4078800a8226b22,Gryffindor,20.0,1.0,4.45,0.005937,0.000000,0.003446,0.011470,0.014586
4,60ad06b911f13500a8aeb5c9,Gryffindor,11.0,1.0,9.77,0.002968,0.000000,0.007566,0.012745,0.014921
5,609ea182c6214800a8b5295c,Gryffindor,10.0,2.0,1.49,0.002639,0.005051,0.001154,0.012812,0.011850
...,...,...,...,...,...,...,...,...,...,...
3341,60b3fdee6a426400a8131f2b,Hufflepuff,227.0,7.0,157.96,0.074208,0.030303,0.122322,0.061787,0.102920
3342,60982190bea2a400a8a72657,Ravenclaw,13.0,1.0,2.73,0.003628,0.000000,0.002114,0.010956,0.012827
3343,609c191bc6214800a8aa0100,Slytherin,55.0,1.0,28.63,0.017480,0.000000,0.022171,0.015257,0.029641
3344,60b3bd3bc7002f00a87aa2f4,Ravenclaw,7.0,1.0,2.03,0.001649,0.000000,0.001572,0.010729,0.011610


In [32]:
total_activity.sort_values('duration_sessions_metric', ascending=False).head(3)[['user_id','duration_sessions_metric']]

,user_id,duration_sessions_metric
2587,60a3eca41ff78a00a9d34774,0.793088
2076,60a7496cb4078800a82fb6a4,0.786219
1244,60ad23ce0ddefb00a870a9c7,0.650460


In [33]:
total_activity.sort_values('duration_actions_metric', ascending=False).head(3)[['user_id','duration_actions_metric']]

,user_id,duration_actions_metric
1244,60ad23ce0ddefb00a870a9c7,0.875025
2076,60a7496cb4078800a82fb6a4,0.786219
2587,60a3eca41ff78a00a9d34774,0.745950


## On question 2
We have created two different metrics in order to capture the level of user activity that comprises of time spent and actions performed. Both metrics designate the same three users as the most active - their difference is in the ranking.

## Let us now turn to question 1

The first question in line, but last as we treat them, is about the Hogwarts school has the most active students.

We will use the metrics we have created for the previous questions.

### ------------COMMENT------------ ###

- We see again the same picture with of a distribution in the family of exponential distributions, where extremely large values dominate. We should then be careful with the use of the mean.

In [34]:
fig9 = px.histogram(total_activity[total_activity['duration_sessions_metric']< 1]
                    , x="duration_sessions_metric"
                    , color="house"
                    , marginal="rug"
                    , hover_data=total_activity.columns
                    )
fig9.show()

In [35]:
fig8 = px.histogram(total_activity[total_activity['duration_actions_metric']< 1]
                    , x="duration_actions_metric"
                    , color="house"
                    , marginal="rug"
                    , hover_data=total_activity.columns)
fig8.show()

In [36]:
## In the presence of large values we chose the median as a more appropriate statistic.

total_activity.\
    groupby('house').\
    median()\
    [['duration_sessions_metric', 'duration_actions_metric']].\
    sort_values('duration_sessions_metric', ascending=False)


,duration_sessions_metric,duration_actions_metric
house,,
Ravenclaw,0.013430,0.016156
Hufflepuff,0.012696,0.015297
Gryffindor,0.012253,0.014515
Slytherin,0.012005,0.014039


## On question 1

Finally, we see that Ravenclaw has the most active students. Both our metrics paint the same picture for all 4 houses.

## Extension

Since we have the data at our disposal, we will give it a try and compute some usual mobile app analytics metrics.

In [37]:
## We start with the number of Daily Active Users

dau_df = \
    combined_users_actions[['action_ts_to_dt', 'user_id']].\
    groupby([combined_users_actions['action_ts_to_dt'].dt.date])\
    .nunique()[['user_id']].rename(columns={'user_id':'daily_user_count'})

dau_df.index.rename('Day', inplace=True)

dau_df.head()

,daily_user_count
Day,
2021-05-05,70
2021-05-06,101
2021-05-07,83
2021-05-08,67
2021-05-09,65


In [38]:
fig10 = dau_df.daily_user_count.plot(
        title='Daily Active Users'
        , y = 'daily_user_count'
        , labels = { 'daily_user_count' : "DAU" }
        )
fig10.show()

fig10.write_image("../presentation/figures/fig10.png")

In [103]:
## Last, we take a look at the Daily Active Users, broken down by house

per_house_dau_df = \
    combined_users_actions[['string_date','user_id', 'house']].\
    groupby(['string_date','house']).nunique().rename(columns={'user_id':'daily_user_count'})

per_house_dau_df.reset_index(level='house', inplace=True)
per_house_dau_df.index.rename('Day', inplace=True)


fig14 = px.line(per_house_dau_df
                , color='house'
                ,title='Activity level: daily total actions over daily total users '
                , y = 'daily_user_count'
                , labels = { 'daily_user_count' : "Activity level by house" }
)

fig14.show()

fig14.write_image("../presentation/figures/fig14.png")

### ------------COMMENT------------ ###

We do not observe any significant differences among the houses, so we'll only present the rest of the analysis for all Hogwarts

In [39]:
## We move on to registrations

daily_reg_df = \
    combined_users_actions[['account_ts_to_dt', 'user_id']].\
    groupby([combined_users_actions['account_ts_to_dt'].dt.date])\
    .nunique()[['user_id']].rename(columns={'user_id':'daily_reg_count'})

daily_reg_df.index.rename('Day', inplace=True)

daily_reg_df.head()

,daily_reg_count
Day,
2021-05-05,81
2021-05-06,82
2021-05-07,72
2021-05-08,63
2021-05-09,52


In [40]:

fig6 = daily_reg_df.daily_reg_count.plot(
        title='Daily Registrations'
        , y = 'daily_reg_count'
        , labels = { 'daily_reg_count' : "Daily Registartions" }        
        )
fig6.show()


fig6.write_image("../presentation/figures/fig6.png")

In [41]:
## Instead of the Monthly users, for lack of extended data, we compute the number of Weekly Active Users

wau_df = \
    combined_users_actions[['action_ts_to_dt', 'user_id']].\
    groupby(pd.Grouper(key='action_ts_to_dt', freq='W-MON'))\
    .nunique()[['user_id']].rename(columns={'user_id':'weekly_user_count'})

wau_df.index.rename('Week', inplace=True)

wau_df.head()

,weekly_user_count
Week,
2021-05-10,444
2021-05-17,711
2021-05-24,892
2021-05-31,943
2021-06-07,825


In [42]:
fig11 = wau_df.weekly_user_count.plot(
        title='Weekly Active Users'
        , y = 'weekly_user_count'
        , labels = { 'weekly_user_count' : "WAU" }
        )
fig11.show()


fig11.write_image("../presentation/figures/fig11.png")

In [43]:
## We move on to stickiness - the degree to which the users "stick around" an app. 
stickiness_df = pd.concat([dau_df, wau_df], axis=1, join="outer").bfill()
stickiness_df.index.rename('Day', inplace=True)
stickiness_df['stickiness'] = (stickiness_df.daily_user_count/stickiness_df.weekly_user_count) * 100

In [44]:

fig12 = stickiness_df.stickiness.plot(
        title='Stickiness: percentage of daily users over weekly users'
        , y = 'stickiness'
        , labels = { 'stickiness' : "Stickiness" }
        )
fig12.show()


fig12.write_image("../presentation/figures/fig12.png")

In [45]:
## Average daily sessions per daily active user (DAU) : Total daily sessions / Total daily active users

dacu_df = \
    combined_users_actions[['action_ts_to_dt', 'action']].\
    groupby([combined_users_actions['action_ts_to_dt'].dt.date,])\
    .count()[['action']].rename(columns={'action':'total_daily_actions'})

dacu_df.index.rename('Day', inplace=True)

dacu_df.head()

,total_daily_actions
Day,
2021-05-05,2153
2021-05-06,3274
2021-05-07,2584
2021-05-08,2044
2021-05-09,2209


In [46]:
daily_sessions_df = pd.merge(dau_df, dacu_df, left_index=True, right_index=True)
daily_sessions_df['activity_level'] = daily_sessions_df.total_daily_actions / daily_sessions_df.daily_user_count
daily_sessions_df.head()

,daily_user_count,total_daily_actions,activity_level
Day,,,
2021-05-05,70,2153,30.757143
2021-05-06,101,3274,32.415842
2021-05-07,83,2584,31.132530
2021-05-08,67,2044,30.507463
2021-05-09,65,2209,33.984615


In [47]:

fig13 = daily_sessions_df.activity_level.plot(
        title='Activity level: daily total actions over daily total users '
        , y = 'activity_level'
        , labels = { 'activity_level' : "Activity level" }
        )
fig13.show()


fig13.write_image("../presentation/figures/fig13.png")

## Let us now pay attention to retention.

We will now present two concepts related to retention and churn. 

The first is a numerical representation of the balance between new users, the ones the app retains in the long run and those that stop using the app. We define a period and we compute the following: 
- (the number of users at end of period – number of users acquired during period) / number of users at start of period ) x 100

In our case the data does not easily lend itself to this kind of analysis, because users start out at zero and then registrations abruptly stop. Therefore we modify the metric in the following way, recall that registration lasted for a month:
- number of users in the last two months - which is after registrations stopped / number of registrations

The second concept is that of survival analysis. We will present data for each user that measure the time interval from registration until last action/session. We have seen this data before; we now present it in a different form that allows to see immediately the proportions of users that have been active for more than a certain period of time.

In [48]:
## Modified retention - cutoff date is a week later than the last registrations
print ( "The active users (at least on action) in the last two calendar months are {} per cent of all registrations".format(
    round(
    combined_users_actions.query('20210612 < action_ts_to_dt').user_id.unique().size \
    / total_activity.user_id.size \
        , 3) *100
))

The active users (at least on action) in the last two calendar months are 8.4 per cent of all registrations


In [49]:
## Max active period from minutes to hours 

survival_data = fig1_data/60

In [106]:
survival_fig = px.ecdf(survival_data
            , ecdfmode="reversed"
            , title="Survival function - Cumulative Probability for variable 'Time diff from Registration to Last Action'  " 
            )

series_names = [ "Time difference in hours" ]

for idx, name in enumerate(series_names):
    survival_fig.data[idx].name = name

survival_fig.show()


survival_fig.write_image("../presentation/figures/survival_fig.png")